In [1]:
import os

In [ ]:
from_dir = "/home/sebastian/masters/data/210916_TCRpMHCmodels/models/"
to_dir = "/home/sebastian/masters/data/neat_data/tcrpmhc/"
model_suffix = "model_TCR-pMHC.pdb"
for subdir in os.listdir(from_dir):
    subdir_id = subdir.split("_")[0]
    new_name = f"tcrpmhc_{subdir_id}.pdb"
    os.system(f"mv {from_dir}/{subdir}/{model_suffix} {to_dir}/{new_name}")

In [ ]:
from_dir = "/home/sebastian/masters/data/embedding_verification/raw_filtered_models"
to_dir = "/home/sebastian/masters/data/neat_data/pmhc/"
model_suffix = "model_pMHC.pdb"
for subdir in os.listdir(from_dir):
    subdir_id = subdir.split("_")[0]
    new_name = f"pmhc_{subdir_id}.pdb"
    os.system(f"mv {from_dir}/{subdir}/{model_suffix} {to_dir}/{new_name}")

In [ ]:
from_dir = "/home/sebastian/masters/data/embedding_verification/raw_filtered_models"
to_dir = "/home/sebastian/masters/data/neat_data/p/"
model_suffix = "model_p.pdb"
for subdir in os.listdir(from_dir):
    subdir_id = subdir.split("_")[0]
    new_name = f"p_{subdir_id}.pdb"
    os.system(f"mv {from_dir}/{subdir}/{model_suffix} {to_dir}/{new_name}")

In [2]:
l = ["p", "pmhc", "tcrpmhc"]

for x in l:
    path = f"/home/sebastian/masters/data/neat_data/raw/{x}/"
    files = os.listdir(path)
    for file in files:
        name = os.path.basename(file)
        name = name.split(".")[0].split("_")[1]
        new_name = f"/home/sebastian/masters/data/neat_data/raw/{x}/{name}_{x}.pdb"
        os.system(f"mv {path}/{file} {new_name}")